In [1]:
import pandas as pd
import numpy as np
import os
import forzawrapper

In [6]:
def get_sign(score):
    if type(score) != str:
        return None
    h, a = int(score[0]), int(score[2])
    if h == a:
        return "X"
    elif h > a:
        return "1"
    else:
        return "2"

files = ["sheets/"+f for f in os.listdir("sheets")]

names = []
preds = []
for f in files:
    names.append('-'.join(f.split("-")[2:]).split(".")[0])
    tmp = pd.read_excel(f)
    preds.append(tmp[tmp.columns[5]][11:59].values.astype(str))

#scrape_df = pd.read_html("https://fbref.com/en/comps/1/schedule/World-Cup-Scores-and-Fixtures")[0]
#score_df = scrape_df[~scrape_df.Home.isnull()]

forza_df = forzawrapper.get_scores()
print(forza_df)

pred_df = pd.DataFrame(np.array(preds).T, columns=names)
df = pd.concat([pred_df, forza_df], axis=1)
#df["Score"] = forza_df['Score'].values
#df["Teams"] = score_df.apply(lambda x: '-'.join(x["Home"].split(" ")[:-1]+x["Away"].split(" ")[1:]), axis=1).values
#df["Correct"] = df["Score"].apply(lambda x: get_sign(x))


                     Kickoff                      Teams Score
0   2022-11-20T16:00:00.000Z           Qatar vs Ecuador   0-2
1   2022-11-21T13:00:00.000Z            England vs Iran   6-2
2   2022-11-21T16:00:00.000Z     Senegal vs Netherlands   0-2
3   2022-11-21T19:00:00.000Z               USA vs Wales   1-1
4   2022-11-22T10:00:00.000Z  Argentina vs Saudi Arabia   1-2
5   2022-11-22T13:00:00.000Z         Denmark vs Tunisia   0-0
6   2022-11-22T16:00:00.000Z           Mexico vs Poland   0-0
7   2022-11-22T19:00:00.000Z        France vs Australia   4-1
8   2022-11-23T10:00:00.000Z         Morocco vs Croatia   0-0
9   2022-11-23T13:00:00.000Z           Germany vs Japan   1-2
10  2022-11-23T16:00:00.000Z        Spain vs Costa Rica   7-0
11  2022-11-23T19:00:00.000Z          Belgium vs Canada   1-0
12  2022-11-24T10:00:00.000Z    Switzerland vs Cameroon   1-0
13  2022-11-24T13:00:00.000Z  Uruguay vs Korea Republic   0-0
14  2022-11-24T16:00:00.000Z          Portugal vs Ghana   3-2
15  2022

,Irannejad-Armin,Filip,Martin-Hunhammar,Sellberg,Kickoff,Teams,Score
0,2,2,X,2,2022-11-20T16:00:00.000Z,Qatar vs Ecuador,0-2
1,X,1,1,1,2022-11-21T13:00:00.000Z,England vs Iran,6-2
2,2,2,2,2,2022-11-21T16:00:00.000Z,Senegal vs Netherlands,0-2
3,1,X,2,2,2022-11-21T19:00:00.000Z,USA vs Wales,1-1
4,1,1,1,1,2022-11-22T10:00:00.000Z,Argentina vs Saudi Arabia,1-2
5,1,1,1,1,2022-11-22T13:00:00.000Z,Denmark vs Tunisia,0-0
6,1,2,X,2,2022-11-22T16:00:00.000Z,Mexico vs Poland,0-0
7,1,1,1,1,2022-11-22T19:00:00.000Z,France vs Australia,4-1
8,2,1,2,2,2022-11-23T10:00:00.000Z,Morocco vs Croatia,0-0
9,1,1,1,1,2022-11-23T13:00:00.000Z,Germany vs Japan,1-2


In [3]:
print("Antal rätt efter", (~df['Correct'].isna()).sum(), "matcher")
names = []
scores = []
percent = []
for c in df.columns[:4]:
    names.append(c)
    scores.append((df[c] == df['Correct']).sum())
    percent.append(str(((df[c] == df['Correct']).sum()/(~df['Correct'].isna()).sum())*100)+"%")
    
table = pd.DataFrame([names, scores, percent]).T
table.columns = ["Namn", "Antal Rätt", "Procent"]
table.sort_values("Antal Rätt", ascending=False).reset_index(drop=True)

Antal rätt efter 17 matcher


,Namn,Antal Rätt,Procent
0,Martin-Hunhammar,10,58.82352941176471%
1,Filip,9,52.94117647058824%
2,Sellberg,8,47.05882352941176%
3,Irannejad-Armin,7,41.17647058823529%
